HUGO ERNESTO CARDENAS ALCARAZ 744893

# Laboratorio: Métodos de búsqueda

En las clases anteriores creaste códigos para realizar búsquedas aleatorias (Simulated Annealing) y búsquedas dirigidas (Optimización Bayesiana). Estos métodos de búsqueda se utilizan para facilitar el proceso de optimización de funciones objetivos compleja y costosas de computar.

En este laboratorio usaremos el dataset de los diferentes tipos de iris, y sus longitudes y anchos de pétalos y sépalos. Utilizaremos un RandomForest para crear un modelo de clasificación y el métrico F1 para decidir cuál es el mejor modelo de acuerdo a lo que tenemos disponible.

1. Carga el dataset de Iris

In [237]:
from sklearn import datasets
X, y = datasets.load_iris(return_X_y=True)

2. Importa el archivo `Bosque.py`.

Este archivo contiene la función `RegresionBosque`, que recibe:
- X: las características independientes
- y: la variable de respuesta
- árboles: cantidad total de árboles
- profundidad de bosque: niveles de profundidad del bosque

Su salida es:
- modelo: El objeto con el modelo ajustado
- f1: El métrico que califica qué tan bueno es el modelo que se ajustó.


In [242]:
import Bosque
modelo, f1 = Bosque.RegresionBosque(X, y, 10, 3)
f1

0.9555555555555556

### Actividad 1:

Inicializa un espacio con 5 muestras en nuestro dominio de variables independientes:
- árboles: números enteros entre 5 y 50.
- profundidad: números enteros entre 2 y 10

Utiliza optimización Bayesiana para encontrar la combinación de árboles y profundidad que **maximice** el métrico F1.

In [433]:
import numpy as np
import matplotlib.pyplot as plt
arboles = np.random.randint(5,51,5)
tree = arboles.reshape([-1,1])
profundidad = np.random.randint(2,11,5)
depth = profundidad.reshape([-1,1])
tree,depth

(array([[10],
        [39],
        [ 5],
        [47],
        [12]]),
 array([[5],
        [5],
        [7],
        [8],
        [3]]))

In [435]:
modelo_1,uno = Bosque.RegresionBosque(X, y, arboles[0], profundidad[0])
modelo_2,dos = Bosque.RegresionBosque(X, y, arboles[1], profundidad[1])
modelo_3,tres = Bosque.RegresionBosque(X, y, arboles[2], profundidad[2])
modelo_4,cuatro = Bosque.RegresionBosque(X, y, arboles[3], profundidad[3])
modelo_5,cinco = Bosque.RegresionBosque(X, y, arboles[4], profundidad[4])


In [436]:
uno, dos, tres, cuatro, cinco

(0.9555555555555556,
 0.9555555555555556,
 0.9555555555555556,
 0.9555555555555556,
 0.9666666666666667)

In [437]:
valores_z = np.array([uno,dos,tres,cuatro,cinco])
valores_z

array([0.95555556, 0.95555556, 0.95555556, 0.95555556, 0.96666667])

In [438]:
X_hyper = np.hstack((tree, depth))

In [440]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

kernel = 1.0*RBF(length_scale = 1)
gp = GaussianProcessRegressor(kernel = kernel, n_restarts_optimizer = 10).fit(X_hyper,valores_z)

x_tree = np.linspace(5,50,1000).reshape([-1,1])
x_profundidad = np.linspace(2,10,1000).reshape([-1,1])
x_hyper = np.hstack((x_tree, x_profundidad))
y_pred, y_std = gp.predict(x_hyper, return_std = True)

#y_pred_low = y_pred - 1.96*(y_std)
y_pred_high = y_pred + 1.96*(y_std)

i_next = np.argmax(y_pred_high)
new_X = np.vstack((X_hyper,x_hyper[i_next]))
new_X

array([[10.,  5.],
       [39.,  5.],
       [ 5.,  7.],
       [47.,  8.],
       [12.,  3.],
       [ 5.,  2.]])

### Actividad 2:

Inicializa 2 vectores con posibles valores para las variables independientes:
- árboles: números enteros entre 5 y 50
- profundidad: números enteros entre 2 y 10

Utiliza el algoritmo de Simulated Annealing que siga el siguiente orden:
- Elige un punto de partida para las variables.
- Selecciona al azar una de las dos para modificarlas.
- Selecciona un elemento al azar de la lista que contiene los posibles valores de esa variable.
- Sigue el algoritmo ($p$ y $q$) para decidir si usas esa combinación nueva o si mantienes la anterior.